In [1]:
%matplotlib inline
import numpy as np
import pylab as pl
import scipy.special as special
from scipy.integrate import quad
from scipy.integrate import nquad

# Set plot parameters to make beautiful plots
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20  
pl.rcParams['font.sans-serif'] = 'serif'
#pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'medium'

pl.rcParams['xtick.major.size'] = 8     
pl.rcParams['xtick.minor.size'] = 4     
pl.rcParams['xtick.major.pad']  = 8     
pl.rcParams['xtick.minor.pad']  = 8     
pl.rcParams['xtick.color']      = 'k'     
pl.rcParams['xtick.labelsize']  = 'medium'
pl.rcParams['xtick.direction']  = 'in'    

pl.rcParams['ytick.major.size'] = 8     
pl.rcParams['ytick.minor.size'] = 4     
pl.rcParams['ytick.major.pad']  = 8     
pl.rcParams['ytick.minor.pad']  = 8     
pl.rcParams['ytick.color']      = 'k'     
pl.rcParams['ytick.labelsize']  = 'medium'
pl.rcParams['ytick.direction']  = 'in'    

In [12]:
#want to plot the integrand in Eq. 60 of the paper
#with filled in values of I(...), delta(yn), 

#constants in natural units
#m = 1.
#c = 1.
theta_e = 10.
#nu_c = 1
#B = 10.
#e = 1


#constants in Gaussian cgs units (values from harmony)
m = 9.1093826e-28
c = 2.99792458e10
e = 4.80320680e-10
h = 6.6260755e-27
k_Bol = 1.380658e-16
B = 30.0
#n_e = 100000.


theta = np.pi/3.
epsilon = 0.01
n_e = 1
nu_c = e * B / (2. * np.pi * m * c)
#nu_c = 1.
nu = 1. * nu_c
nu_s = (2./9.) * nu_c * theta_e**2 * np.sin(theta)

#cos_xi = np.arange(-1., 1., 2./100)
#cos_xi = 0.5
#cos_xi = (gamma * nu - n_e * nu_c) / (gamma * nu * beta * np.cos(theta))

#cos_xi = np.arange(-1, 1, 0.1)
#gamma = np.arange(1, 11, 0.1)

#def delta_function(n, gamma):
#    beta = np.sqrt(1. - 1./gamma**2.)
#    cos_xi = (gamma * nu - n_e * nu_c) / (gamma * nu * beta * np.cos(theta))
#    argument = (n * nu_c)/gamma - nu * (1 - beta * cos_xi * np.cos(theta))
#    f = (1/(epsilon * np.sqrt(np.pi))) * np.exp(-argument**2./epsilon**2.)
#    return f


n_dn = (nu / nu_c) * abs(np.sin(theta))
#beta_here = np.sqrt(1. - 1./((1. + theta_e)**2)) 
#ans = (theta_e + 1.) * (nu/nu_c) * (1. - beta_here**2 * (np.cos(theta))**2)
C = 10.

#def n_up(gamma):
#    beta = np.sqrt(1. - 1./gamma**2.)
#    ans = 10.* (theta_e + 1. + (2. * theta_e * nu/nu_c)**(1./3.)) * (nu/nu_c) * (1. - beta**2 * (np.cos(theta))**2)
#    return ans
    

In [10]:
def K_s(n, gamma):
    beta = np.sqrt(1. - 1./gamma**2.)
    cos_xi = (gamma * nu - n * nu_c) / (gamma * nu * beta * np.cos(theta))
    M = (np.cos(theta) - beta * cos_xi)/np.sin(theta)
    N = beta * np.sqrt(1. - cos_xi**2)
    z = (nu * gamma * beta * np.sin(theta) * np.sqrt(1. - cos_xi**2))/nu_c
    K_xx = M**2 * (special.jv(n, z))**2
    K_yy = N**2 * (special.jvp(n, z, 1))**2
    ans = K_xx + K_yy
    return ans

def gamma_up_limit(n):
    gamma_up_nume = (n * nu_c /nu) + np.absolute(np.cos(theta)) * np.sqrt((n * nu_c /nu)**2 - (np.sin(theta))**2)
    gamma_deno = (np.sin(theta))**2
    gamma_up = gamma_up_nume / gamma_deno
    return gamma_up

def gamma_dn_limit(n):
    gamma_dn_nume = (n * nu_c /nu) - np.absolute(np.cos(theta)) * np.sqrt((n * nu_c /nu)**2 - (np.sin(theta))**2)
    gamma_deno = (np.sin(theta))**2
    gamma_dn = gamma_dn_nume / gamma_deno
    return gamma_dn

def Operator_D_gamma(gamma):
    prefactor = (np.pi * nu / (m * c**2)) * (n_e / (theta_e * special.kn(2., 1./theta_e)))
    body = (- 1./theta_e) * np.exp(- gamma / theta_e)
    f = prefactor * body
    return f

gamma_here_1 = np.sqrt((1. + theta_e) * (2. * theta_e * nu/nu_c)**(1./3.))
beta_here_1  = np.sqrt(1. - 1./(gamma_here_1**2))
n_peak_first_part = (theta_e + 1. + (2. * theta_e * nu/nu_c)**(1./3.))
n_peak_second_part = (nu/nu_c) * (1. - (beta_here_1 * np.cos(theta))**2)
n_peak       = n_peak_first_part * n_peak_second_part


def absorptivity_I_62(n, gamma):
    
    beta = np.sqrt(1. - 1./gamma**2.)
    cos_xi = (gamma * nu - n_e * nu_c) / (gamma * nu * beta * np.cos(theta))
    
    prefactor = - c * e**2 / (2. * nu)
    body_1 = gamma**2 * beta
    body_2 = Operator_D_gamma(gamma) * K_s(n, gamma)
    f = prefactor * body_1 * body_2
    return f

def absorptivity_integrand_62(n, gamma):
    beta = np.sqrt(1. - 1./gamma**2.)
    prefactor = 1./(nu * beta * abs(np.cos(theta)))
    f = absorptivity_I_62(n, gamma) * prefactor
    return f

def anu_n_integral_62_one(n):
    
    def absorptivity_integrand_62_once(gamma):
        jnu_integrand_62 = absorptivity_integrand_62(n, gamma)
        return jnu_integrand_62
    
    ans_once = quad(absorptivity_integrand_62_once, gamma_dn_limit(n), gamma_up_limit(n), epsabs = 0, epsrel = 1e-8)[0]
    
    return ans_once

def anu_62_integral_from_n_max_to_n_limit(n_max, n_dn):
    
    if(n_max < n_dn):
        n_max = n_dn
    
    ans = quad(anu_n_integral_62_one, n_max, C * n_peak, epsabs = 0, epsrel = 1e-8)[0]
    
    return ans
    

def anu_62_sum_to_n_max_integral_to_n_limit(n_max_sum):
    ans = 0
    for i in range(int(n_dn + 1.), int(1 + n_dn + n_max_sum)):
        ans = ans + anu_n_integral_62_one(i)
#        print ans
        
    total = ans + anu_62_integral_from_n_max_to_n_limit(n_max_sum, n_dn)
    return total



In [11]:
n_max_sum = 30.

print anu_62_sum_to_n_max_integral_to_n_limit(n_max_sum)

8.57856082973e-13


In [14]:
T_e = theta_e * m * c**2 / k_Bol
def Planck_function():
    prefactor = 2. * h * nu**3 / (c**2)
    tail = 1. / (np.exp(h * nu / (k_Bol * T_e)) - 1.)
    ans = prefactor * tail
    return ans

print Planck_function() * 8.57856082973e-13

1.10252089831e-22
